# Part 1

#### Import necessary modules

In [140]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim 
import folium
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Use BeautifulSoup to scrap the webpage and find the required table 

In [21]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')

#### Copy the contents of the table to three separate arrays corresponding to the three columns

In [22]:
A, B, C = [], [], []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) == 3:
        A.append(str(cells[0].find(text=True)).strip())
        B.append(str(cells[1].find(text=True)).strip())
        C.append(str(cells[2].find(text=True)).strip().replace(' / ', ', '))


#### Convert the arrays to dataframe

In [23]:
df = pd.DataFrame(A, columns=['PostalCode'])
df['Borough'] = B
df['Neighborhood'] = C
#df.replace(r'\s+|\\n', ' ', regex=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop all the rows containing 'Not assigned' Boroughs

In [24]:
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames , inplace=True)
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Check for duplicates in the PostalCode so that Neighborhoods can be aggregated, if duplicates are present

In [25]:
 df['PostalCode'].duplicated().any()

False

#### ==> Since there are no duplicates, and Neighborhoods are already aggregated no rearrangemets are required

#### The code to make rearrangements anyway is as follows

In [26]:
result = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)
result.head()

,,Neighborhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Check for 'Not assigned' or empty Boroughs

In [27]:
df.loc[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


#### ==> All the Boroughs are assigned, so no rearrangements are required

#### The shape of the final dataframe

In [28]:
print(df.shape)

(103, 3)


# Part 2

In [29]:
df_lat_lon = pd.read_csv('Geospatial_Coordinates.csv')
df_lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df_lat_lon.columns = ['PostalCode', 'Latitude', 'Longitude']
df_lat_lon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
df_cord= pd.merge(df, df_lat_lon[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode')
df_cord

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3

#### Selecting only boroughs having the name Toronto

In [55]:
df_toronto = df_cord[df_cord.Borough.str.contains('Toronto',case=False)]
print(df_toronto.head())
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]))

   PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031  
The dataframe has 4 boroughs and 39 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [51]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [59]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [61]:
CLIENT_ID = '#########################' # your Foursquare ID
CLIENT_SECRET = '#########################' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [67]:
LIMIT = 100
radius = 500

In [62]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [101]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        while True:
            # make the GET request
            # print(requests.get(url).json()["response"], '\n\n')
            try:
                results = requests.get(url).json()["response"]["groups"][0]["items"]
            except Exception:
                print('Retrying API call...')
                continue
            break
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [102]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Retrying API call...
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Retrying API call...
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airpo

#### Size of the resulting dataframe

In [106]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Venues per Neighborhood

In [108]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,18,18,18,18,18,18
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


In [110]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


## Analyze Each Neighborhood

In [112]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Neighborhood with top 5 venues

In [115]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3     Cheese Shop  0.04
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.13
1         Coffee Shop  0.09
2      Breakfast Spot  0.09
3         Yoga Studio  0.04
4  Italian Restaurant  0.04


----Business reply mail Processing CentrE----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2     Comic Shop  0.06
3    Pizza Place  0.06
4     Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4             Plane  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.20
1                 Café  0.06
2   Italian Restaurant  0.06
3       S

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Italian Restaurant,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Stadium,Bakery,Intersection,Italian Restaurant,Burrito Place,Climbing Gym
2,Business reply mail Processing CentrE,Yoga Studio,Burrito Place,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Restaurant,Pizza Place,Smoke Shop,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Boutique,Bar,Coffee Shop,Plane
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Middle Eastern Restaurant,Fried Chicken Joint,Thai Restaurant,Salad Place


## Cluster Neighborhoods

In [134]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [138]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Toronto Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Distribution Center,Event Space,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0,Coffee Shop,Sushi Restaurant,Diner,Creperie,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Bookstore,Hotel
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0,Café,Coffee Shop,Cocktail Bar,Gastropub,Italian Restaurant,American Restaurant,Hotel,Cosmetics Shop,Creperie,Lingerie Store
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0,Pub,Coffee Shop,Trail,Health Food Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store


In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [144]:
toronto_merged.loc[toronto_merged['Toronto Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Distribution Center,Event Space,Electronics Store
4,Downtown Toronto,0,0,Coffee Shop,Sushi Restaurant,Diner,Creperie,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café
9,Downtown Toronto,0,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Bookstore,Hotel
15,Downtown Toronto,0,0,Café,Coffee Shop,Cocktail Bar,Gastropub,Italian Restaurant,American Restaurant,Hotel,Cosmetics Shop,Creperie,Lingerie Store
19,East Toronto,0,0,Pub,Coffee Shop,Trail,Health Food Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
20,Downtown Toronto,0,0,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Italian Restaurant,Farmers Market
24,Downtown Toronto,0,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Middle Eastern Restaurant,Fried Chicken Joint,Thai Restaurant,Salad Place
25,Downtown Toronto,0,0,Grocery Store,Café,Park,Candy Store,Restaurant,Baby Store,Nightclub,Gas Station,Coffee Shop,Athletics & Sports
30,Downtown Toronto,0,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,American Restaurant,Hotel,Clothing Store,Gym,Pizza Place
31,West Toronto,0,0,Pharmacy,Bakery,Brewery,Bank,Bar,Supermarket,Café,Athletics & Sports,Music Venue,Pizza Place


#### Cluster 2

In [146]:
toronto_merged.loc[toronto_merged['Toronto Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,1,1,Park,Bus Line,Swim School,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 3

In [148]:
toronto_merged.loc[toronto_merged['Toronto Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,2,Health & Beauty Service,Garden,Pool,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster4

In [150]:
toronto_merged.loc[toronto_merged['Toronto Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,3,3,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 5

In [152]:
toronto_merged.loc[toronto_merged['Toronto Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Toronto Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,4,4,Park,Trail,Playground,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
91,Downtown Toronto,4,4,Park,Trail,Playground,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
